> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.11 - Data Model

## Exercícios

#### Q1. `dataclass`
Exercício adaptado de [codechalleng.es/bites/154/](https://codechalleng.es/bites/154/) e [codechalleng.es/bites/320/](https://codechalleng.es/bites/320/).

Neste desafio, você deve escrever uma `dataclass` chamada `Bite` que gerencia 3 atributos: `number`, `title` e `level`. Seus tipos são:
* `number` - `int`, 
* `title` - `str`, 
* `level` -  classe `Enum` chamada `BiteLevel` com os atributos `Beginner`, `Intermediate`, `Advanced`. 

Exemplo de dado: `{'number': 154, 'title': 'Escreva uma dataclass', 'level': BiteLevel.Intermediate}`

As características dessa classe são:
* O atributo`level` tem um valor padrão `BiteLevel.Beginner`
* Uma coleção de objetos `Bite` tem que ser ordenável somente pelo atributo `number`
* Implemente o método especial `__str__` para imprimir o Bite na forma `f'{number} - {title} ({level})'`

Teste sua classe executando o seguinte código:
```python
bites = []
bites.append(Bite(154, 'Escreva uma dataclass', 'Intermediate'))
bites.append(Bite(1, 'Some n valores'))
bites.append(Bite(37, 'Reescreva um loop com recursão', 'Intermediate'))

for b in bites.sort(): print(b)
# Ordem esperada na saída:
# 1 - Some n valores (Beginner)
# 37 - Reescreva um loop com recursão (Intermediate)
# 154 - Escreva uma dataclass (Intermediate)
```

In [ ]:
from dataclasses import dataclass
from enum import Enum
from typing import List

class BiteLevel(Enum):
    Beginner = "Beginner"
    Intermediate = "Intermediate"
    Advanced = "Advanced"

@dataclass(order=True)
class Bite:
    number: int
    title: str
    level: BiteLevel = BiteLevel.Beginner
    
    def __post_init__(self):
        if isinstance(self.level, str):
            self.level = BiteLevel[self.level]
    
    def __str__(self):
        return f'{self.number} - {self.title} ({self.level.value})'

# Testes
bites = []
bites.append(Bite(154, 'Escreva uma dataclass', 'Intermediate'))
bites.append(Bite(1, 'Some n valores'))
bites.append(Bite(37, 'Reescreva um loop com recursão', 'Intermediate'))

bites.sort()

for b in bites:
    print(b)

#### Q2. `Pydantic`
> Adaptada desse [tutorial de Pydantic](https://github.com/adonath/scipy-2023-pydantic-tutorial/tree/main) criado por [Axel Donath](https://github.com/adonath) e [Nick Langellier](https://github.com/nlangellier).

Observe a seguinte lista de observações da previsão do tempo em Murmansk, Russia.
```python
data_samples = [
    {
        "date": "2023-05-20",
        "temperature": 62.2,
        "isCelsius": False,
        "airQualityIndex": "24",
        "sunriseTime": "01:26",
        "sunsetTime": "00:00",
    },
    {
        "date": "2023-05-21",
        "temperature": "64.4",
        "isCelsius": "not true",
        "airQualityIndex": 23,
        "sunriseTime": "01:10",
        "sunsetTime": "00:16",
    },
    {
        "date": "2023-05-22",
        "temperature": 14.4,
        "airQualityIndex": 21,
    },
]
```

Escreva um script que calcule e imprima a temperatura média (em Celsius) em Murmansk para as datas fornecidas. Em seu script, você deve incluir um modelo Pydantic que registre com sucesso todos os elementos dados. Note que:

* Algumas amostras estão faltando dados. Você deve decidir quando o atributo pode ter um valor padrão ou quando definí-lo como opcional (`typing.Optional`). 
* Você precisará implementar pelo menos um validador de campo para transformar atributos. Dica: teste primeiro quais vão falhar :)



In [ ]:
from pydantic import BaseModel, Field, validator
from typing import Optional
from datetime import datetime
from statistics import mean

class WeatherObservation(BaseModel):
    date: datetime = Field(..., description="Date in YYYY-MM-DD format")
    temperature: float
    isCelsius: bool = True
    airQualityIndex: Optional[int] = None
    sunriseTime: Optional[str] = None
    sunsetTime: Optional[str] = None
    
    @validator('date', pre=True)
    def parse_date(cls, v):
        if isinstance(v, str):
            return datetime.strptime(v, "%Y-%m-%d")
        return v
    
    @validator('temperature', pre=True)
    def parse_temperature(cls, v):
        if isinstance(v, str):
            return float(v)
        return v
    
    @validator('isCelsius', pre=True)
    def parse_is_celsius(cls, v):
        if isinstance(v, str):
            return v.lower() in ['true', '1', 'yes', 't']
        if isinstance(v, bool):
            return v
        return True
    
    @validator('airQualityIndex', pre=True)
    def parse_air_quality(cls, v):
        if isinstance(v, str):
            return int(v)
        return v
    
    @property
    def temperature_celsius(self):
        """Converte temperatura para Celsius se necessário"""
        if self.isCelsius:
            return self.temperature
        else:
            return (self.temperature - 32) * 5/9

# Dados de exemplo
data_samples = [
    {
        "date": "2023-05-20",
        "temperature": 62.2,
        "isCelsius": False,
        "airQualityIndex": "24",
        "sunriseTime": "01:26",
        "sunsetTime": "00:00",
    },
    {
        "date": "2023-05-21",
        "temperature": "64.4",
        "isCelsius": "not true",
        "airQualityIndex": 23,
        "sunriseTime": "01:10",
        "sunsetTime": "00:16",
    },
    {
        "date": "2023-05-22",
        "temperature": 14.4,
        "airQualityIndex": 21,
    },
]

# Valida e processa os dados
observations = []
for data in data_samples:
    obs = WeatherObservation(**data)
    observations.append(obs)

# Calcula temperatura média em Celsius
temperatures_celsius = [obs.temperature_celsius for obs in observations]
avg_temp = mean(temperatures_celsius)

print(f"Temperatura média em Murmansk: {avg_temp:.1f}°C")
print("\nDetalhes:")
for obs in observations:
    print(f"{obs.date.date()}: {obs.temperature_celsius:.1f}°C")



    

#### Q3
> Adaptada desse [tutorial de Pydantic](https://github.com/adonath/scipy-2023-pydantic-tutorial/tree/main) criado por [Axel Donath](https://github.com/adonath) e [Nick Langellier](https://github.com/nlangellier).

Na célula a seguir, coletamos dados reais de uma das principais APIs de previsão do tempo, [open-meteo](https://open-meteo.com/en/docs). Não se preocupe em entender esse código, o mais importante é entender o resultado que ele retorna, ilustrado a seguir para uma coleta da temperatura dos últimos 15 dias em Itabira -MG. Caso deseje alterar a cidade de coleta, basta alimentar a latitude e longitude desejada, como nas opções a seguir.
* Itabira: `'latitude': -19.656655787605846, 'longitude': -43.228922960534476`
* Bom Despacho: `'latitude': -19.726308457732443, 'longitude': -45.27462803349767`

```python
{
  "latitude": -19.5,
  "longitude": -43.375,
  "generationtime_ms": 0.01800060272216797,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 2.0,
  "hourly_units": {
    "time": "iso8601",
    "temperature_2m": "\u00b0C"
  },
  "hourly": {
    "time": [
      "2024-07-19T00:00",
      "2024-07-19T01:00",
      "2024-07-19T02:00",
      ...
    ],
    "temperature_2m": [
      21.9,
      20.9,
      20.0,
      ... 
    ]
  }
}
```

Você deve escrever um modelo Pydantic `OpenMeteo` que receba diretamente a resposta dessa API, através do comando:
```python
dados = OpenMeteo(**response)
``` 

Para comportar a estrutura hierárquica desse dicionário (é um dicionário com alguns dicionários internos), você deve criar uma classe Pydantic para cada dicionário interno (`HourlyUnits` e `Hourly`), com seus respectivos atributos. Essas classes serão atributos da classe principal `OpenMeteo`, que terá também os outros atributos da resposta (`latitude`, `longitude`, etc.).



In [ ]:
import requests, json

url = 'https://api.open-meteo.com/v1/forecast'
lat, long = -19.656655787605846, -43.228922960534476
params = {'latitude': lat, 'longitude': long, 'elevation': 2,
          'hourly': 'temperature_2m', 'forecast_days': 15}
response = requests.get(url, params=params).json()
print(json.dumps(response, indent=2))

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

# Usando os dados já validados da Q3
# Se não tiver os dados, pode usar este código para coletar novamente:
if 'dados' not in locals():
    url = 'https://api.open-meteo.com/v1/forecast'
    lat, long = -19.656655787605846, -43.228922960534476
    params = {'latitude': lat, 'longitude': long, 'elevation': 2,
              'hourly': 'temperature_2m', 'forecast_days': 15}
    response = requests.get(url, params=params).json()
    dados = OpenMeteo(**response)

# Preparar dados para o gráfico
times = [datetime.fromisoformat(t.replace('Z', '+00:00')) for t in dados.hourly.time]
temperatures = dados.hourly.temperature_2m

# Criar o gráfico
plt.figure(figsize=(12, 6))
plt.plot(times, temperatures, marker='o', linewidth=1, markersize=3, color='#2E86AB')

# Formatação do gráfico
plt.title('Temperatura Horária em Itabira - MG (Últimos 15 dias)', fontsize=14, pad=20)
plt.xlabel('Data e Hora', fontsize=12)
plt.ylabel('Temperatura (°C)', fontsize=12)

# Rotacionar labels do eixo x para melhor legibilidade
plt.xticks(rotation=45)
plt.gca().set_xlim(times[0], times[-1])

# Formatar grid e estilo
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Adicionar algumas estatísticas
avg_temp = np.mean(temperatures)
min_temp = np.min(temperatures)
max_temp = np.max(temperatures)

plt.text(0.02, 0.98, f'Média: {avg_temp:.1f}°C | Mín: {min_temp:.1f}°C | Máx: {max_temp:.1f}°C', 
         transform=plt.gca().transAxes, fontsize=10, verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.show()

# Informações adicionais
print(f"Temperatura média: {avg_temp:.1f}°C")
print(f"Temperatura mínima: {min_temp:.1f}°C")
print(f"Temperatura máxima: {max_temp:.1f}°C")
print(f"Período: {times[0].strftime('%d/%m/%Y %H:%M')} a {times[-1].strftime('%d/%m/%Y %H:%M')}")

#### Q4. 

Com os dados carregados na questão anterior plote um gráfico de linha, com a biblioteca de sua preferência, onde o eixo `x` são os timestamps (data e hora) e o eixo `y` é a temperatura medida.

In [ ]:
import seaborn as sns
import pandas as pd

df = pd.DataFrame({
    'timestamp': times,
    'temperature': temperatures
})

sns.set_style("whitegrid")

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='timestamp', y='temperature', 
             linewidth=2, color='steelblue', marker='o', markersize=2)

plt.title('Temperatura Horária - Itabira, MG', fontsize=16, fontweight='bold')
plt.xlabel('Data e Hora', fontsize=12)
plt.ylabel('Temperatura (°C)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()